In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import json
import matplotlib.pyplot as plt
from friendlyfit.plotting import bandcolorf

with open('products/walkers.json', 'r') as f:
    data = json.loads(f.read())

# Find best scoring walker
max_score = max([data[x]['score'] for x in data])
for x in data:
    if data[x]['score'] == max_score:
        data = data[x]
        break
band_names = list(set(data['bands']))

fig = plt.figure(figsize=(12,6))
for band in band_names:
    xs, ys = zip(*[
        [x, y] for x, y, z in
        zip(data['times'], data['model_magnitudes'], data['bands'])
        if z == band])
    plt.plot(xs, ys, c=bandcolorf(band))
    xs, ys, yes = zip(*[
        [x, y, ye] for x, y, ye, z in
        zip(data['times'], data['magnitudes'], data['e_magnitudes'], data['bands'])
        if z == band])
    plt.errorbar(xs, ys, yerr=yes, c=bandcolorf(band), fmt='o', label=band)
plt.gca().invert_yaxis()
plt.margins(0.1,0.1)
plt.legend()
plt.show()
fig.savefig('products/lc.png')